In [ ]:
import os 
import sys
from pathlib import Path
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
images_path = Path("/content/drive/MyDrive/Projects/Food-101/food_101_data/images")
food_names = os.listdir(images_path)
len(food_names)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim 
from torchvision import models, transforms

In [ ]:
class FoodDataset(Dataset):
  def __init__(self, food_names):
    self.X = []
    self.Y = []

    self.preprocess = transforms.Compose([
      transforms.Resize(224),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    number_foods = len(food_names)
    food2idx = { name:idx for idx, name in enumerate(food_names) }

    IMG_SIZE = 128
    for idx, food_name in enumerate(food_names):
      food_path = images_path/food_name
      food_idx = food2idx[food_name]
      
      print(f"Preparing {food_name}: {idx+1}/{number_foods}")

      for img_name in tqdm(os.listdir(food_path)):
        image_path = food_path/img_name
        
        img = Image.open(str(image_path))

        try:
          img = self.preprocess(img)
          
          self.X.append(img)
          self.Y.append(food_idx)
        except: 
          pass

    self.Y = np.array(self.Y)
  
  def one_hot_encode(self, idx, length):
    onehot = np.zeros((length, 1))
    onehot[idx] = 1
    return onehot

  def __len__(self):
    assert len(self.X) == len(self.Y)
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [ ]:
test_food_names = [
  "chicken_wings",
  "churros",
  "donuts",
  "french_fries",
  "hamburger",
  "hot_dog",
  "ice_cream",
  "lasagna",
  "cheesecake",
  "apple_pie"
]

In [ ]:
food_dataset = FoodDataset(test_food_names)
len(food_dataset), food_dataset[0][0].shape, food_dataset[0][1].shape

In [ ]:
import pickle 
with open(images_path/"../test_food_dataset", "wb") as f:
  pickle.dump(food_dataset, f)

In [ ]:
import pickle
with open(images_path/"../test_food_dataset", "rb") as f:
  food_dataset = pickle.load(f)
len(food_dataset), food_dataset[0][0].shape, food_dataset[0][1].shape

In [ ]:
dataset = DataLoader(food_dataset, batch_size=64, shuffle=True)
dataset

## Transfer Learning

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
  since = time.time()
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f"Epoch: {epoch}/{num_epochs-1}")
    print("-" * 10)

    n_epoch = 0
    running_loss = 0.0
    running_corrects = 0.0

    # Iterate over data
    for inputs, labels in dataloader:
      inputs = inputs.to(device).float()
      labels = labels.to(device)

      # Zero the parameters of the gradients
      optimizer.zero_grad()

      n_epoch += inputs.size()[0]

      # Forward Propagation
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      # Backward Propagation + Optimize 
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / n_epoch
    epoch_acc = running_corrects.double() / n_epoch

    print("Loss: {:.4f} Acc: {:.4f}".format(epoch_loss, epoch_acc))

  time_elapsed = time.time() - since
  print(f"Training complete in {time_elapsed // 60}m {time_elapsed % 60}")

  return model

In [ ]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"
device = torch.device(dev)

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
 
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model_ft = train_model(model_ft, dataset, criterion, optimizer_ft, num_epochs=5)

## Own training

In [ ]:
class Foodnet(nn.Module):
  def __init__(self, n_classes):
    super(Foodnet, self).__init__()
    
    # Vgg19
    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, (3,3), padding=1),
      nn.Conv2d(64, 64, (3,3), padding=1)
    )
    self.maxpool1 = nn.MaxPool2d((2,2), stride=2)

    self.conv2 = nn.Sequential(
      nn.Conv2d(64, 128, (3,3), padding=1),
      nn.Conv2d(128, 128, (3,3), padding=1)    
    )
    self.maxpool2 = nn.MaxPool2d((2,2), stride=2)

    self.conv3 = nn.Sequential(
      nn.Conv2d(128, 256, (3,3), padding=1),
      nn.Conv2d(256, 256, (3,3), padding=1),
      nn.Conv2d(256, 256, (3,3), padding=1),
      nn.Conv2d(256, 256, (3,3), padding=1)
    )
    self.maxpool3 = nn.MaxPool2d((2,2), stride=2)
  
    self.conv4 = nn.Sequential(
      nn.Conv2d(256, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1)
    )
    self.maxpool4 = nn.MaxPool2d((2,2), stride=2)

    self.conv5 = nn.Sequential(
      nn.Conv2d(512, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1),
      nn.Conv2d(512, 512, (3,3), padding=1)
    )
    self.maxpool5 = nn.MaxPool2d((2,2), stride=2)

    self.fc1 = nn.Linear(25088, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, n_classes)

  def forward(self, X):
    X = self.conv1(X)
    X = self.maxpool1(X)

    X = self.conv2(X)
    X = self.maxpool2(X)
    
    X = self.conv3(X)
    X = self.maxpool3(X)

    X = self.conv4(X)
    X = self.maxpool4(X)

    X = self.conv5(X)
    X = self.maxpool5(X)

    X = torch.flatten(X, 1)

    X = self.fc1(X)
    X = self.fc2(X)
    X = self.fc3(X)

    return X

In [ ]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)

net = Foodnet(len(test_food_names)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
opt = optim.Adam(net.parameters(), lr=0.000001)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  epoch_loss = 0.
  epoch_acc = 0
  n_epoch = 0

  for x_batch, y_batch in dataset:
    x_batch = x_batch.to(device).float()
    y_batch = y_batch.to(device).long()

    out = net(x_batch)

    if y_batch.size()[0] != 64:
      continue

    net.zero_grad()
    loss = criterion(out, y_batch)
    
    loss.backward()20
    opt.step()

    batch_acc = out.argmax(dim=1)
    epoch_acc += (batch_acc == y_batch).sum().float()

    epoch_loss += loss.item()
    n_epoch += x_batch.size()[0]

  epoch_loss /= n_epoch
  epoch_acc /= n_epoch
  print(f"EPOCH: {epoch+1}/{EPOCHS} - loss: {epoch_loss} - acc: {epoch_acc*100}") 